In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCTC, AutoProcessor
import torchaudio.functional as F
import soundfile
import librosa
import time

from IPython.display import Audio

In [ ]:
model_id = "patrickvonplaten/wav2vec2-large-xlsr-53-spanish-with-lm"


model = AutoModelForCTC.from_pretrained(model_id)
processor = AutoProcessor.from_pretrained(model_id)

model = model.cuda()

In [ ]:
audio_path = "/data/podcasts/El hilo/La-ultima-clinica-de-aborto-en-la-frontera-sur-de-Texas.mp3"
wav, sr = librosa.load(audio_path)

wav_clip = wav[:sr*60]
Audio(wav_clip, rate=sr)

In [ ]:
resampled_wav = F.resample(torch.tensor(wav_clip), sr, 16_000).numpy()

input_values = processor(resampled_wav, return_tensors="pt").input_values
input_values = input_values.cuda()
start = time.time()
with torch.no_grad():
    logits = model(input_values).logits
transcription = processor.batch_decode(logits.cpu().numpy()).text
print(transcription)
print(time.time() - start)

### Notes

on RTX 3090 it takes less then 1 sec ~ 830 msec to transcribe 1 min of audio. Pretty good!